# llm-movie-recommender

Denne notebooken demonstrerer bruk av en språkmodel (Language Model - LM) og en vektordatabase for anbefaling av filmer.

Denne metoden løser det såkalte "cold start" problemet for anbefalings-anbefalingsalgoritmer, hvor det ikke finnes
data fra brukeren av systemet eller data fra andre brukere av systemet til å gi anbefalinger.

I tilfellet hvor det finnes data på hvilke brukere som foretrekker hvilke filmer, er det enklere å lage en anbefalingsalgoritme.
Der er det mulig å bruke denne dataen til å gi anbefalinger - for eksempel, blandt alle brukere som har gitt høy vurdering (rating)
for "Star Wars", er det også gitt høy rating for "Back to The Future".

I stedet, kan vi bruke en språkmodell til å finne film-anbefalinger basert på hvor likt sammendraget av filmen er, og basert på hvor ofte
navnet på filmen forekommer i samme kontekst som andre filmer.

Denne anbefalingsalgoritmen er egentlig en clustering-metode. Datasettet består av unlabelled data - informasjon om filmer, uten informasjon
om hvilke filmer som er like eller hvilke brukere som like hvilke filmer. En clustering-algoritme gir informasjon om hvor like filmer er hverandre,
og kan dermed bruker til å gi anbefalinger mer. For eksempel kan vi forvente (håpe) på at filmer som handler å reise i tid ved hjelp av tidsmaskiner
ligger i samme kluster - det vil si at cluster-algoritmen mener disse filmene har en likhet med hverandre.

La oss først begynne med å hente filmdata fra den åpne filmdatabasen TMDb ved å bruke API-et.

Dette er veldig rett fram. Vi henter informasjon om alle tilgjengelige filmer som har 1000 brukervurderinger eller mer. 

In [ ]:
from config import TMDB_API_KEY
import requests

tmdb_url = "https://api.themoviedb.org/3"

def fetch_top_rated_movies(min_votes: int = 1_000) -> list[dict]:
    """
    Fetch top rated movies from themoviedb

    Parameters
    ----------
    min_votes
        Only fetch movies that have this number of votes or more

    Returns
    -------
    A list of movies, each movie in a dictionary
    """
    # Set the API endpoint and parameters
    url = tmdb_url + "/discover/movie"

    # Fetch movies page by page until there are no more
    movies = []
    i = 1
    while True:
        params = {
            "api_key": TMDB_API_KEY,
            "sort_by": "vote_average.desc",
            "vote_count.gte": min_votes,
            "page": i,
        }

        # Send a GET request to the API and raise an exception if it fails
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Extract the movie data from the response
        movies_ = response.json()["results"]
        if not movies_:
            break
        movies.extend(movies_)
        i += 1

    return movies


In [15]:
movies = fetch_top_rated_movies()

API-et returnerer grunnleggende informasjon om filmene. Av interesse her er tittel, sjanger, år, og sammendrag.


In [18]:
print(movies[0])

{'adult': False, 'backdrop_path': '/zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg', 'genre_ids': [18, 80], 'id': 278, 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Imprisoned in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 121.768, 'poster_path': '/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg', 'release_date': '1994-09-23', 'title': 'The Shawshank Redemption', 'video': False, 'vote_average': 8.7, 'vote_count': 26258}



Her er følgende av interesse:
- Tittel
- Filmsjanger
- Sammendrag
- År

Vi samler denne informasjonen i en dataframe for enklere og raskere prosessering.
Vi bruker `polars` som et raskere og mer moderne alternativ til `pandas` selv om det har lite å bety i denne sammenheng
siden datasettet er så lite.

Først henter vi filmsjangernavn fra API-et og bruker disse istedet for filmsjanger-ID'er.

Deretter samler vi informasjonen vi ønsker å bruke til å sammenligne i en kolonne ("text") - nemlig  tittel, år, sammendrag og sjangere.

In [63]:
import polars as pl
from functools import lru_cache

@lru_cache
def get_id_to_genre() -> dict[int, str]:
    """Return a mapping from genre id to genre name"""
    # Fetch genre names
    url = tmdb_url + "/genre/movie/list"
    params = {"api_key": TMDB_API_KEY}
    response = requests.get(url, params=params)
    response.raise_for_status()
    id_and_genre = response.json()["genres"]

    # Map genre ids to genre names
    id_to_genre = {x["id"]: x["name"] for x in id_and_genre}

    return id_to_genre


def prep_movies(movies: list[dict]) -> pl.DataFrame:
    """
    Prepare movie data for embedding

    Attributes
    ----------
    movies
        List of movies

    Returns
    -------
    polars.DataFrame
    """
    # Create a Polars DataFrame from the movie data
    df = pl.DataFrame(movies)

    id_to_genre = get_id_to_genre()
    # Add genre names from genre ids
    df_genres = df["genre_ids"].map_elements(
        lambda li: [id_to_genre[id_] for id_ in li], return_dtype=pl.List(pl.String)
    )
    df = df.with_columns(df_genres.alias("genres"))

    # Extract year from release date
    df = df.with_columns(df["release_date"].str.head(4).alias("year"))

    # Merge all information into a single text column
    text = (
        "Movie title: "
        + df["title"]
        + ".\nYear: "
        + df["year"]
        + ".\nOverview: "
        + df["overview"]
        + "\nGenres: "
        + df["genres"].list.join(", ")
        + "."
    )

    df = df.with_columns(text.alias("text"))

    # Drop unecessary columns
    df = df[["id", "title", "year", "overview", "genres", "text"]]

    # The API may sometimes yield duplicates. Keep only unique rows
    df = df.unique(subset="id", maintain_order=True)

    # The vector database expects IDs in string format
    df = df.with_columns(df["id"].cast(str))
    return df

In [64]:
df = prep_movies(movies)

La oss se på innholdet til DataFramen

In [65]:
len(df)

4119

In [66]:
df.head(1)

id,title,year,overview,genres,text
str,str,str,str,list[str],str
"""278""","""The Shawshank …","""1994""","""Imprisoned in …","[""Drama"", ""Crime""]","""Movie title: T…"


Dette gir følgende tekst for filmen. Det er denne teksten vi skal bruke i vektordatabasen til å finne lignende filmer.

In [67]:
print(df[0, "text"])

Movie title: The Shawshank Redemption.
Year: 1994.
Overview: Imprisoned in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.
Genres: Drama, Crime.


Vi instansierer en vektordatabase som vi skal bruke til å finne liknende dokumenter. Her finnes det mange alternativer, og vi har valgt ChromaDB hvor vi kan velge en valgfri embedding-funksjon.

In [32]:
import chromadb
client = chromadb.Client()

In [71]:
title_collection = client.get_or_create_collection("movie_titles")

In [ ]:
if not title_collection.count():
    title_collection.add(documents=df["title"].to_list(), ids=df["id"].to_list())

In [46]:
results = title_collection.query(query_texts=["Star Wars"], n_results=20)

In [48]:
print(results["documents"])

[['Star Wars', 'Star Wars: The Last Jedi', 'Star Wars: The Force Awakens', 'Star Wars: The Clone Wars', 'Star Wars: The Rise of Skywalker', 'Star Wars: Episode I - The Phantom Menace', 'Starship Troopers', 'Rogue One: A Star Wars Story', 'Return of the Jedi', 'Empire of the Sun', 'Guardians of the Galaxy', 'Solo: A Star Wars Story', 'Star Trek', 'The Empire Strikes Back', 'Star Wars: Episode III - Revenge of the Sith', 'Interstellar', 'Guardians of the Galaxy Vol. 2', 'King Kong', 'King Kong', 'Star Wars: Episode II - Attack of the Clones']]


For å demonstrere at LMen gir større likhet mellom dokumenter av lik semantisk betydning, legger vi til dokumentet "Star Alliance". I motsetning til tidligere dokumenter, er ikke dette en film, men navn på et selskap som er en en allianse av flyselskaper.

In [50]:
title_collection.add(documents=["Star Alliance"], ids=["9999"])

Databasen tilbyr flere alternativer for å måle likheten mellom dokumenter. Likheten mellom to dokumenter er gitt av distansen mellom de to tilhørende vektor embeddingene. Som default bruker ChromaDB $L^2$ distanse, som er gitt av:

$ d=∑_i(A_i - B_i)^2 $

Hvor $A_i$ er index $i$ i vektor embeddingen til dokument $A$

In [51]:
results = title_collection.query(query_texts=["Star Wars"], n_results=20)

In [52]:
print(results["documents"])

[['Star Wars', 'Star Wars: The Last Jedi', 'Star Wars: The Force Awakens', 'Star Wars: The Clone Wars', 'Star Wars: The Rise of Skywalker', 'Star Wars: Episode I - The Phantom Menace', 'Starship Troopers', 'Rogue One: A Star Wars Story', 'Return of the Jedi', 'Empire of the Sun', 'Guardians of the Galaxy', 'Solo: A Star Wars Story', 'Star Trek', 'The Empire Strikes Back', 'Star Wars: Episode III - Revenge of the Sith', 'Star Alliance', 'Interstellar', 'Guardians of the Galaxy Vol. 2', 'King Kong', 'King Kong']]


Som vi ser så dukker ikke "Star Alliance" opp som et lignende resultat til "Star Wars". LMen kjenner ikke dette navnet som en film, og det har dermed mye lavere likhet enn navnet på andre 

In [54]:
title_collection.query(query_texts=["Star Wars"], n_results=1, include=["embeddings"])["embeddings"]

[[[-0.0732547789812088,
   0.010070987045764923,
   -0.0035544871352612972,
   -0.013902664184570312,
   -0.04536248371005058,
   0.010174545459449291,
   0.06956266611814499,
   0.029881272464990616,
   0.0726681649684906,
   0.04335178807377815,
   0.018892085179686546,
   -0.016079893335700035,
   0.024354442954063416,
   -0.010189964435994625,
   -0.014761065132915974,
   -0.017748933285474777,
   0.03628627955913544,
   -0.036271754652261734,
   0.033778801560401917,
   -0.05755142495036125,
   0.01229142677038908,
   0.05022326484322548,
   0.032130636274814606,
   0.01590275764465332,
   0.0214584618806839,
   0.08383522182703018,
   0.003329054219648242,
   -0.00615691626444459,
   -0.10529165714979172,
   -0.10661821812391281,
   0.022368373349308968,
   0.034402817487716675,
   -0.0833054631948471,
   0.04602351412177086,
   -0.05026637017726898,
   -0.04611540958285332,
   0.00021591233962681144,
   0.013707726262509823,
   0.024085164070129395,
   -0.021809205412864685,
   

In [ ]:
An embedding represents the semantic essence of a document. 

Andre eksempler hvor bruker av LM og vektordatabase kan være nyttig for å finne like items (for eksemepel dokumenter)